In [54]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [55]:
# Endpoint
sparql = SPARQLWrapper("https://shared.semantics.cancer.gov/sparql")

In [56]:
# Query for Alzheimer's (C2866)
# Core properties:
# - ncit:P97: DEFINITION
# - ncit:P90: FULL_SYN (Synonyms)
# - rdfs:label: Preferred Name
# Additional interoperability properties:
# - ncit:P106: Semantic_Type
# - ncit:P207: UMLS_CUI
query_string = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?prefLabel ?synonym ?definition ?semanticType ?umlsCUI ?icd10Code ?meshCode ?conceptStatus
WHERE {
  ncit:C2866 ncit:P97 ?definition .  # P97 = Definition
  ncit:C2866 rdfs:label ?prefLabel .
  OPTIONAL { ncit:C2866 ncit:P90 ?synonym }  # P90 = Synonym
  OPTIONAL { ncit:C2866 ncit:P106 ?semanticType }  # P106 = Semantic_Type
  OPTIONAL { ncit:C2866 ncit:P207 ?umlsCUI }  # P207 = UMLS_CUI
}
GROUP BY ?prefLabel ?definition ?semanticType ?umlsCUI ?icd10Code ?meshCode ?conceptStatus
"""

In [57]:
sparql.setQuery(query_string)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [58]:
# 3. Print Results
print("=" * 120)
print("CORE PROPERTIES")
print("=" * 120)
print(f"{'Preferred Name':<30} | {'Synonyms':<40} | {'Definition'}")
print("-" * 120)

# Group results by core properties to avoid duplicates
seen_combinations = set()
for result in results["results"]["bindings"]:
    pref_label = result["prefLabel"]["value"]
    definition = result["definition"]["value"]
    synonym = result.get("synonym", {}).get("value", "N/A")
    key = (pref_label, definition, synonym)
    if key not in seen_combinations:
        seen_combinations.add(key)
        print(f"{pref_label:<30} | {synonym:<40} | {definition}")

print("\n" + "=" * 120)
print("INTEROPERABILITY PROPERTIES")
print("=" * 120)

# Print interoperability properties (should be same across all rows)
if results["results"]["bindings"]:
    first_result = results["results"]["bindings"][0]
    semantic_type = first_result.get("semanticType", {}).get("value", "N/A")
    umls_cui = first_result.get("umlsCUI", {}).get("value", "N/A")
    
    print(f"Semantic Type (P106): {semantic_type}")
    print(f"UMLS CUI (P207): {umls_cui}")


CORE PROPERTIES
Preferred Name                 | Synonyms                                 | Definition
------------------------------------------------------------------------------------------------------------------------
Alzheimer's Disease            | Alzheimer's Dementia                     | A progressive, neurodegenerative disease characterized by loss of function and death of nerve cells in several areas of the brain leading to loss of cognitive function such as memory and language.
Alzheimer's Disease            | Alzheimer Disease                        | A progressive, neurodegenerative disease characterized by loss of function and death of nerve cells in several areas of the brain leading to loss of cognitive function such as memory and language.
Alzheimer's Disease            | Alzheimer's Disease                      | A progressive, neurodegenerative disease characterized by loss of function and death of nerve cells in several areas of the brain leading to loss of cogni